In [ ]:
from pyspark.sql import SparkSession

# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.executor.memory", "5g")
    .config("spark.driver.memory", "5g")
    .getOrCreate()
)

In [2]:
import pandas as pd
import geopandas as gpd
import folium
import matplotlib.pyplot as plt
import os


vic_suburbs = gpd.read_file("../data/map/LOCALITY_POLYGON.shp")
crime_df = pd.read_parquet('../data/curated/crimes.parquet')



In [ ]:
crime_df.to_csv('../data/curated/crimes.csv')

In [3]:
crime_df = pd.read_csv('../data/curated/crimes.csv')

In [4]:
# Rename 'GAZLOC' to gazetted locality
vic_suburbs['suburb'] = vic_suburbs['GAZLOC'].str.lower()

# Filter the dataframe to only include the gazetted localities and their geometries
vic_suburbs_filtered = vic_suburbs[['suburb', 'geometry']]

crime_df.head()


,Unnamed: 0,year,suburb,offence_division,total_offence_count,prev_year_offence_count,difference_change
0,0,2014-12-31,abbeyard,1,0,NaN,NaN
1,1,2015-12-31,abbeyard,1,0,0.0,0.0
2,2,2016-12-31,abbeyard,1,0,0.0,0.0
3,3,2017-12-31,abbeyard,1,0,0.0,0.0
4,4,2018-12-31,abbeyard,1,2,0.0,2.0


In [4]:
vic_suburbs_filtered.head()


,suburb,geometry
0,mollongghip,"POLYGON ((2417354.185 2445906.94, 2417260.386 ..."
1,north blackwood,"POLYGON ((2445160.214 2452798.651, 2445239.392..."
2,basalt,"POLYGON ((2421860.482 2465074.51, 2421860.053 ..."
3,llanelly,"POLYGON ((2394406.798 2527031.044, 2394351.249..."
4,murrabit west,"POLYGON ((2397518.057 2666586.745, 2397393.211..."


In [7]:
merged_df = pd.merge(crime_df, vic_suburbs_filtered, how='inner', on='suburb')
merged_df = gpd.GeoDataFrame(merged_df, geometry='geometry')

merged_2015 = merged_df[merged_df["year"] == "2015-12-31"]
merged_2015_divsion2 = merged_2015[merged_2015["offence_division"] == 2]

merged_2015_divsion2['geometry'] = merged_2015_divsion2['geometry'].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")




/home/msetty/.local/lib/python3.10/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [12]:
import geopandas as gpd
import folium
from folium.plugins import HeatMap
import pandas as pd

# Convert the DataFrame to a GeoDataFrame if not already
gdf_2015 = gpd.GeoDataFrame(merged_2015, geometry='geometry')

# Calculate the centroids for each polygon to get representative points
gdf_2015['centroid'] = gdf_2015['geometry'].centroid

# Extract the latitude (y) and longitude (x) coordinates of the centroids
heat_data = [
    [row['centroid'].y, row['centroid'].x, row['difference_change']]
    for _, row in gdf_2015.iterrows() if not pd.isna(row['difference_change'])
]

# Create a base map, centered on coordinates (modify as needed)
m = folium.Map(location=[-37.8136, 144.9631], zoom_start=8)

# Add the heatmap layer
HeatMap(heat_data, min_opacity=0.5, max_val=max(gdf_2015['difference_change']), radius=15, blur=10).add_to(m)

# Display the map
m


/tmp/ipykernel_84571/1410675552.py:12: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_2015['centroid'] = gdf_2015['geometry'].centroid
/tmp/ipykernel_84571/1410675552.py:24: UserWarning: The `max_val` parameter is no longer necessary. The largest intensity is calculated automatically.
  HeatMap(heat_data, min_opacity=0.5, max_val=max(gdf_2015['difference_change']), radius=15, blur=10).add_to(m)


In [8]:
import geopandas as gpd
import folium
from folium.plugins import HeatMap

# Assuming your DataFrame is named 'merged_2015' and contains a 'geometry' column with polygons

# Convert the DataFrame to a GeoDataFrame if not already
gdf = merged_2015_divsion2

# Create a base map, centered on coordinates (modify as needed)
m = folium.Map(location=[-37.8136, 144.9631], zoom_start=8)

# Extract centroids and difference_change values for the heatmap
heat_data = [
    [row.geometry.centroid.y, row.geometry.centroid.x, row['difference_change']]
    for _, row in gdf.iterrows() if not pd.isna(row['difference_change'])
]

# Add the heatmap layer
HeatMap(heat_data, min_opacity=0.5, max_val=max(gdf['difference_change']), radius=15, blur=10).add_to(m)

# Display the map
m


/tmp/ipykernel_104431/691145358.py:20: UserWarning: The `max_val` parameter is no longer necessary. The largest intensity is calculated automatically.
  HeatMap(heat_data, min_opacity=0.5, max_val=max(gdf['difference_change']), radius=15, blur=10).add_to(m)
